# Moroccan Darija Trigger Word Classification ed .2

# IMPORTS

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from keras.layers import Dense,Conv1D,Flatten,MaxPool1D,Dropout,LSTM
from keras.models import Sequential

import librosa #if not installed install with : !pip install librosa 

# Loading the data

In [14]:
TRAINPATH= r"C:\Users\ahmed\Desktop\trigerwords\data\train"

In [15]:
TESTPATH= r"C:\Users\ahmed\Desktop\trigerwords\data\test"

# Preprocessing the data

In [16]:
train_file_names = tf.io.gfile.listdir(str(TRAINPATH))
print(train_file_names)

['0mbsexwoja8u4n12v5ql.wav', '0p5dwnmxgabovt8u4hrf.wav', '0tjkrsnv1lfcub7hp5ex2.wav', '0y4whrmeixdalkz8s6bju.wav', '1t6w3k05q7j8clfmy9ug.wav', '2glnftjbpehdy7xi953a.wav', '2jgkywshetpr43xdabn7.wav', '32bixqwuv5anmtgzl8cj.wav', '3kp695hfniwozetu0aqrg.wav', '3xpvqgtyd9ucwmk6bof4.wav', '3zebh15sl6djpx4ri7g0u.wav', '41g2s8z3i7dkhfwtyoul0.wav', '43kjhxb2d9qpmatrv51e.wav', '46khm1wtnip8zeb7doj0.wav', '4eh591vt6x2surm08l3o.wav', '4nt1978i5lmdk36pz2yxh.wav', '4pg0yrcdhjvo73mtanxf.wav', '4q67nahyc3gm2xerd9su.wav', '4qktn8a5f0jcbev1irldx.wav', '5dqlz0onixs9phy2vwm1b.wav', '5e842oanxf379huk0g6c.wav', '5iaso8rzd7pfexyvuk1jn.wav', '5vnh0ky62o9d4sm8xbgzt.wav', '5wt2u8abqh3lds4k9ivj.wav', '64gqyv7n3ksmirj0lp9t1.wav', '67crbeaq5diwjmtvpzxs0.wav', '6fbgkadenil574two32h.wav', '6fu2bqt759is1yh38vo0.wav', '6zub4xg3ij7daqf91lwyc.wav', '71srdp82j6xick4ovl9z.wav', '7rni6fcob13metld9auv.wav', '7rvnxekopwi81b3c6fsd.wav', '7xbwc2u3o8fiyhgrzqsv1.wav', '7xyjhfudqg81bz2lot36k.wav', '83nitwe2cp5f1bjr07zs.wav', '85z

In [10]:
test_file_names = tf.io.gfile.listdir(str(TESTPATH))

In [41]:

def audio_to_array(file , semipath ):
    x,_ = librosa.load(semipath +r"\\" +file, sr=3000)
    x=(x-min(x))/(max(x)-min(x))
    return x.astype(numpy.float16)

def get_label(file ):


In [36]:
audio_to_array("ulcdyxnbz6jqh9wmk4pa.wav" ,TRAINPATH)

array([0.505 , 0.506 , 0.504 , ..., 0.4028, 0.5474, 0.5605], dtype=float16)

In [42]:
get_label("ulcdyxnbz6jqh9wmk4pa.wav" ) 

1

In [21]:
train_csv=pd.read_csv(r"C:\Users\ahmed\Desktop\trigerwords\train.csv")

In [23]:
train_csv.head()

,id,label
0,gudj6np7lx4ov5a9y3wq,1
1,mi2z51y4jb7thxuf8ldr,1
2,ziuab84tvf07snxjhyodp,0
3,ulcdyxnbz6jqh9wmk4pa,1
4,gj8rfxzae4vibtohk62n1,0


In [256]:
train_csv['label'].isna().sum()

0

In [28]:
train_csv['id'] =train_csv['id'].astype(str)+'.wav'
train_csv['id']

0       gudj6np7lx4ov5a9y3wq.wav
1       mi2z51y4jb7thxuf8ldr.wav
2      ziuab84tvf07snxjhyodp.wav
3       ulcdyxnbz6jqh9wmk4pa.wav
4      gj8rfxzae4vibtohk62n1.wav
                 ...            
139     pyjvr51h8lea7364w0io.wav
140    nkpyoaxbzi4ftv62cd98g.wav
141     97fic3rxubmq0np82dzs.wav
142     fznl3vs67m1i2praw5c9.wav
143     7rni6fcob13metld9auv.wav
Name: id, Length: 144, dtype: object

In [32]:
test_csv=pd.read_csv(r"C:\Users\ahmed\Desktop\trigerwords\test.csv")

In [34]:
test_csv['id'] =test_csv['id'].astype(str)+'.wav'

In [45]:
max_length = 0
for file in train_file_names :
    if len(audio_to_array(file , TRAINPATH ))>max_length:
        max_length= len(audio_to_array(file,TRAINPATH ))
max_length

8960

In [134]:
x_train=[]
y_train=[]
c=0
for file in train_file_names :
    x_train.append(audio_to_array(file , TRAINPATH))
    c+=1
    for aud in train_csv['id'] :
        if file ==  aud :
            y_train.append(train_csv.loc[train_csv["id"] == aud , "label"])
            
        
    

In [51]:
x_test=[]
for audio in test_file_names :
    x_test.append(np.pad(audio_to_array(audio , TESTPATH),(0,max_length-len(audio_to_array(audio , TESTPATH))),'constant').reshape(1,max_length))

In [137]:
x_train=np.array(x_train)
x_test=np.array(x_test)

In [107]:
x_train

array([array([0.521, 0.521, 0.521, ..., 0.   , 0.   , 0.   ], dtype=float16),
       array([0.4092, 0.4092, 0.4092, ..., 0.    , 0.    , 0.    ], dtype=float16),
       array([0.5835, 0.5835, 0.5835, ..., 0.    , 0.    , 0.    ], dtype=float16),
       array([0.573, 0.573, 0.573, ..., 0.   , 0.   , 0.   ], dtype=float16),
       array([0.5093, 0.5093, 0.5093, ..., 0.    , 0.    , 0.    ], dtype=float16),
       array([0.4966, 0.4966, 0.4966, ..., 0.    , 0.    , 0.    ], dtype=float16),
       array([0.485, 0.485, 0.485, ..., 0.   , 0.   , 0.   ], dtype=float16),
       array([0.422, 0.422, 0.422, ..., 0.   , 0.   , 0.   ], dtype=float16),
       array([0.558, 0.558, 0.558, ..., 0.   , 0.   , 0.   ], dtype=float16),
       array([0.5425, 0.5425, 0.5425, ..., 0.    , 0.    , 0.    ], dtype=float16),
       array([0.5605, 0.5605, 0.5605, ..., 0.    , 0.    , 0.    ], dtype=float16),
       array([0.4685, 0.4685, 0.4685, ..., 0.    , 0.    , 0.    ], dtype=float16),
       array([0.4717, 

In [135]:
for i in range(len(x_train)):
    x_train[i]=np.pad(x_train[i],(0,max_length-len(x_train[i])),'constant')

In [141]:
x_train.shape

(144, 8960)

In [164]:
x_train=x_train.reshape(-1,8960,1)

In [145]:
l=[]
for i in range(len(y_train)):
    if y_train[i][0]==1:
        l.append(np.array([0,1]))
    else :
        l.append(np.array([1,0]))

In [146]:
l=np.array(l)
y_train=l

# Building our model

In [167]:

model = models.Sequential()
model.add(Conv1D(filters=256, kernel_size=5, padding='same', activation='relu',
                         input_shape=x_train.shape[1:]))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPool1D(pool_size=4))

model.add(Flatten())
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_22 (Conv1D)          (None, 8960, 256)         1536      
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 2240, 256)        0         
 g1D)                                                            
                                                                 
 conv1d_23 (Conv1D)          (None, 2236, 128)         163968    
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 559, 128)         0         
 g1D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 71552)             0         
                                                                 
 dense_17 (Dense)            (None, 64)              

In [168]:

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

In [238]:
ACCURACY_THRESHOLD=0.92
class myCallback(tf.keras.callbacks.Callback):
	def on_epoch_end(self, epoch, logs={}):
		if(logs.get('acc') > ACCURACY_THRESHOLD):
			print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))
			self.model.stop_training = True

callbacks = myCallback()

# Training the model

In [169]:
EPOCHS=30
model.fit(x_train,y_train,epochs=EPOCHS,batch_size=32,verbose=1)

Epoch 1/30
5/5 [==============================] - 5s 784ms/step - loss: 0.7138 - accuracy: 0.5139
Epoch 2/30
5/5 [==============================] - 4s 785ms/step - loss: 0.7021 - accuracy: 0.5278
Epoch 3/30
5/5 [==============================] - 4s 772ms/step - loss: 0.6925 - accuracy: 0.5347
Epoch 4/30
5/5 [==============================] - 4s 769ms/step - loss: 0.6907 - accuracy: 0.4861
Epoch 5/30
5/5 [==============================] - 4s 782ms/step - loss: 0.6863 - accuracy: 0.5486
Epoch 6/30
5/5 [==============================] - 4s 781ms/step - loss: 0.6825 - accuracy: 0.5694
Epoch 7/30
5/5 [==============================] - 4s 804ms/step - loss: 0.6813 - accuracy: 0.6042
Epoch 8/30
5/5 [==============================] - 4s 771ms/step - loss: 0.6779 - accuracy: 0.5694
Epoch 9/30
5/5 [==============================] - 4s 770ms/step - loss: 0.6677 - accuracy: 0.6528
Epoch 10/30
5/5 [==============================] - 4s 791ms/step - loss: 0.6529 - accuracy: 0.6667
Epoch 11/30
5/5 [==

In [170]:
model.save(r"C:\Users\ahmed\Desktop\trigerwords")

INFO:tensorflow:Assets written to: C:\Users\ahmed\Desktop\trigerwords\assets


# Evaluating the model

In [172]:
predictions=model.predict(x_test)

In [173]:
predictions=predictions.reshape((82, 2))

In [174]:
pred=predictions[:,1]

In [175]:
for i in range(len(pred)):
    if pred[i]>=0.5:
        pred[i]=int(0)
    else :
        pred[i]=int(1)

In [176]:
final_pred=pred.astype(int)

In [177]:
final_pred

array([1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0])

# FOR SUBMISSION ON KAGGLE 

In [ ]:
#test=pd.read_csv(r"C:\Users\ahmed\Downloads\sample_submission.csv")

In [184]:
#for index , row in test.iterrows() :
#    test["label"].loc[index]=final_pred[index]

In [182]:
#test.to_csv(r"C:\Users\ahmed\Documents\sample_submission10.csv",index=False)